# Assignment  Practice Text classification with Naive Bayes 

## Notebook made by   (If not filled in correctly: 0 pts for assignment)

__Name(s)__: Jonas van Oenen, Steven van Beek

__Student id(s)__ : 10670947, 10292527

### Pledge (taken from [Coursera's Honor Code](https://www.coursera.org/about/terms/honorcode) )

<img src='https://imgur.com/FZ1EJGU'/>
<img src='https://imgur.com/iKMnTpf'/>

# Practice Text classification with Naive Bayes  
        
<ol>
  <li>Normalize the values for "ministerie" and choose 10 ministeries to work with. </li>
  <li>Implement the two algorithms in Fig MRS.13.2, using your earlier code for creating term and document frequencies.
  It might be easier to use the representation and formula given in MRS section 13.4.1.</li>
  <li>On this collection, train NB text classifiers for 10 different classes with enough and interesting data.</li>
  <li>Compute for each term and each of your 10 classes its utility for that class using mutual information.</li>
  <li>For each class, show the top 10 words as in Figure 13.7 in MRS.</li>
  <li>Evaluate your classifiers using Precision, Recall and F1. (
       <br/>
      Give a table in which you show these values for using the top 10, top 100 terms and all terms, for all of your 10 classes.
      Thus do feature selection per class, and use for each class the top n best features for that class. 
      <br/>
  Also show the microaverage(s) for all 10 classes together.
  <br/>
  If you like you can also present this in a figure like MRS.13.8. 
  Then compute the F1 measure for the same number of terms as in that figure.</li>
<li> You have done the complete implementation by yourself. Congratulations! You can also use `scikit-learn` routines for all of this work. Do that. So follow [this text classification tutorial](http://scikit-learn.org/stable/tutorial/text_analytics/working_with_text_data.html)  and implement the same steps but now with your kamervragen dataset. Also use [mutual information feature selection](http://scikit-learn.org/stable/modules/feature_selection.html) to select the K-best features, and compare the results as before.
</li>
  <li>Reflect and report briefly about your choices in this process and about the obtained results. Also reflect on the differences between the scikit learn approach and the "own implementation approach".</li>
</ol>

<h3>Training/Testing</h3>
<p>It is important that you do not test your classifier using documents that have also been used in training.
    So split up your collection in a training set and a test set. A 80%-20% split is reasonable.

<br/>
    If you have too little data you can use 5 or <a href="http://en.wikipedia.org/wiki/Cross-validation_(statistics)#k-fold_cross-validation">10-fold cross validation</a>.</p>

<!--
<h2>Form of presentation</h2>
<ul>
    <li>Make slides or wikipages and have your system running (this could just be an IPython notebook with a classify function that accepts any string and classifies it.) ~~and be able to accept documents from the web.~~ </li>
    <li>Create one or two slides or wikipages for each of the sub exercises listed above.
</li>
<li>Make it clear in the heading of the slides which sub exercises you talk about.</li>
    <li>Show running code with one or two  good examples (a TP of course, but also a FP and an error-analysis is nice to show). </li>
</ul>
-->

<h2>Form of handing in your final product</h2>

* An IPython notebook with for each question, a MarkDown cell containing the question, a code cell which solves the question, an output cell with the output, followed by a MarkDown cell with explanation/reflection  

### 1. Normalizing and choosing 10 ministeries

This first code block imports all  needed libraries and defines the 10 chosen ministeries. Throughout the notebook we will use some static variables indicated by a preceding "_".

In [5]:
from collections import defaultdict
from collections import Counter
import nltk
import math
import csv
from __future__ import division
import pandas as pd
import time
from nltk.corpus import stopwords
import numpy as np

_ministeries = ['volksgezondheid', 'binnenlands', 'financ', 'justitie', 
                'econom', 'buitenlands', 'onderwijs', 'verkeer', 'social', 'landbouw']
_csvFile = 'http://maartenmarx.nl/teaching/zoekmachines/LectureNotes/MySQL/KVR1000.csv.gz'

### 2. Term and document frequency

In [6]:
# Statics for the stop words which we filter
_dutchStop= stopwords.words('dutch')

# Names for the csv file
_names=['jaar', 'partij','titel','vraag','antwoord','ministerie']

# Index of our term and document frequency
_index_dataset = {}

# Class frequencies
_classes = {}
        
# Function which checks if a ministery is in our chosen set
def normalize_ministerie(ministerie):
    try:
        for mini in _ministeries:
            if  mini in ministerie:
                return mini

    except Exception as e:
        # there are some munistry which cannot be classified and we catch the exception here
        pass
    return 'foutief'
        
# Function which indexes our dataset and calculates term and document frequency
def index_dataset(kvrdf):
    for mini in _ministeries:
        tic = time.time()
        
        # Get all data per ministry
        miniRows = kvrdf.loc[kvrdf['ministerie'] == mini]
        
        # Store frequency of class
        _classes[mini] = len(miniRows)
        
        # Bag all text together
        text = '\n'.join(list(miniRows.vraag) + list(miniRows.antwoord) + list(miniRows.titel) + list(miniRows.partij))
        
        # Tokenize the text, ignoring stopwords and alpha numericals
        tokenizedText = [w for w in nltk.word_tokenize(text.lower()) if w.isalpha() and not w in set(_dutchStop)]
        
        # Create a counter of all terms in the text
        BoW = Counter(tokenizedText)
        
        # Store counter
        _index_dataset[mini] = BoW
        
        print('Finished indexing '+mini+' ministry data in: ', time.time() - tic)

### 3. Naive Bayes

This code block contains 3 functions. A training function which calculates probabilities for our training set. A classifier function which runs a piece of text against our training set and classifies it as a specific class. The last function is a test function which splits the data in a training and test set, runs the classifier and prints a confusion matrix of the result.

In [7]:
# Probabilities for classes
_classes_probs = {}

# Probabilities for terms
_terms_probs = {}

# Naive bayes trainer which calculates probabilites on our indexed set
def naive_bayes_trainer():
    
    # Calculate prior clas probabilities
    for mini in _classes:
        _classes_probs[mini] = _classes[mini]/sum(_classes.values())
    
    # Create dict with term keys of all ministeries and the total occurences
    allTerms = Counter()
    for d in _index_dataset.values():
        allTerms += d

    # Go through all terms
    for term in allTerms:
        _terms_probs[term] = {}
        
        # Calculate the probability for the term for each class
        for mini in _index_dataset:
            if term in _index_dataset[mini]:
                tcf = (_index_dataset[mini][term] + 1) / (allTerms[term] + len(allTerms))
            else:
                tcf = 1 / (allTerms[term] + len(allTerms))
                
            _terms_probs[term][mini] = tcf
            
# Function which classifies a text based on our naive bayes trained set
def naive_bayes_classifier(text):
    # Tokenize the text exactly the same as in training
    tokenizedText = [w for w in nltk.word_tokenize(text.lower()) if w.isalpha() and not w in set(_dutchStop) and w in _terms_probs]
    
    # Calculate the probabililty of the text belonging to a class for all classes
    probs = {}
    for mini in _classes_probs:
        # Calculate prob of text belonging to this class
        prob = 0.0
        for term in tokenizedText:
            prob += math.log(_terms_probs[term][mini])
        
        probs[mini] = math.log(_classes_probs[mini]) + prob
    
    # Return the arg max of our probabilities
    return max(probs, key = probs.get)

# Function which runs a test on our naive bayes
def naive_bayes_test():
    print('Running Naive Bayes classifier with csv file: ', _csvFile)
    
    # Read CSV file
    totalFile = pd.read_csv(_csvFile, 
                   compression='gzip', sep='\t', 
                   index_col=0, names=_names,
                      )
    # normalize
    totalFile.ministerie = totalFile.ministerie.str.lower().apply(normalize_ministerie)
    totalFile = totalFile.loc[totalFile['ministerie'] != 'foutief']
    
    # create mask
    msk = np.random.rand(len(totalFile)) < 0.8

    # create training and test set
    train = totalFile[msk]
    test = totalFile[~msk]
    
    # Index the data
    index_dataset(train)
    
    # Train our naive bayes
    naive_bayes_trainer()
    
    # Create a confusion matrix
    confusionMatrix = np.zeros((10,10))
    
    # Run the test set on our training data
    for index, row in test.iterrows():
        text = row['vraag']+row['antwoord']+row['titel']+row['partij']
        ministerie = row['ministerie']

        resultClass = naive_bayes_classifier(text)
        
        x = _ministeries.index(ministerie)
        y = _ministeries.index(resultClass)
        
        confusionMatrix[x,y] += 1
        
    
    print('')
    print('Confusion matrix for Naive Bayes Classifier')
    print(confusionMatrix)
        
    
naive_bayes_test()

Running Naive Bayes with csv file:  http://maartenmarx.nl/teaching/zoekmachines/LectureNotes/MySQL/KVR1000.csv.gz
Finished indexing volksgezondheid ministry data in:  1.1503596305847168
Finished indexing binnenlands ministry data in:  0.4575493335723877
Finished indexing financ ministry data in:  0.34320640563964844
Finished indexing justitie ministry data in:  1.2367050647735596
Finished indexing econom ministry data in:  0.36176323890686035
Finished indexing buitenlands ministry data in:  0.5643298625946045
Finished indexing onderwijs ministry data in:  0.5140118598937988
Finished indexing verkeer ministry data in:  0.7073700428009033
Finished indexing social ministry data in:  0.5055696964263916
Finished indexing landbouw ministry data in:  0.37381744384765625

Confusion matrix for Naive Bayes Classifier
[[25.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 2.  0.  0.  4.  0.  0.  0.  0.  0.  0.]
 [ 6.  0.  0.  5.  0.  0.  0.  0.  0.  0.]
 [ 2.  0.  0. 38.  0.  0.  0.  0.  0.  0.]
 [ 5.  0.

### 4. Mutual information
    